In [59]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

from tqdm import tqdm_notebook as tqdm
import os
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

# Define params and helper

In [45]:
vanila_data_path = './data/train/'
aug_data_path = './data/aug/'
test_data_path = './data/test/'
aug_list = ['blur', 'rotation', 'mixup']
num_classes = 12

# Load data

## Vanila dataset

In [51]:
x_train = []
y_train = []
for idx in tqdm(range(num_classes)):
    vanila_data_path_by_class = vanila_data_path + '/' + str(idx) + '/'
    files = os.listdir(vanila_data_path_by_class)
    for file in files:
        x_train.append(np.array(load_img(vanila_data_path_by_class + file)))
        y_train.append(idx)
        
print('num of vanilla samples is ' + str(len(x_train)))

print(x_train[0].shape)

num of vanilla samples is 1518
(224, 224, 3)


## Augmented dataset

In [47]:
for aug_type in aug_list:
    print(aug_type)
    aug_path_by_type = aug_data_path + aug_type + '/'
    for idx in tqdm(range(num_classes)):
        aug_data_path_by_class = aug_path_by_type + str(idx) + '/'
        files = os.listdir(aug_data_path_by_class)
        for file in files:
            x_train.append(np.array(load_img(aug_data_path_by_class + file)))
            y_train.append(idx)

blur


rotation


mixup


In [48]:
print('num of samples after loaded augmented data is ' + str(len(x_train)))

num of samples after loaded augmented data is 4236


# Load test data

In [49]:
x_test = []
y_test = []
for idx in tqdm(range(num_classes)):
    test_data_path_by_class = test_data_path + str(idx) + '/'
    files = os.listdir(test_data_path_by_class)
    for file in files:
        x_test.append(np.array(load_img(test_data_path_by_class + file)))
        y_test.append(idx)
        
print('num of test samples is ' + str(len(x_test)))

num of test samples is 380


# Define model

In [57]:
base_model = ResNet50(weights='imagenet')
x = base_model.output
new_output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=new_output)

# Set trainable layers in model
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


# Train

In [ ]:
model_checkpoint = ModelCheckpoint('./keras.model', save_best_only=True, monitor='acc', mode = 'max', verbose=1)
epochs = 10
batch_size = 32

history = model.fit(np.array(x_train), np.array(y_train),
                    validation_data=[np.array(x_test), np.array(y_test)], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[model_checkpoint],
                    shuffle=True,
                    verbose=1)

Train on 1518 samples, validate on 380 samples
Epoch 1/10
1504/1518 [============================>.] - ETA: 4:29 - loss: 2.4845 - acc: 0.093 - ETA: 2:16 - loss: 2.4852 - acc: 0.093 - ETA: 1:32 - loss: 2.4846 - acc: 0.093 - ETA: 1:09 - loss: 2.4846 - acc: 0.093 - ETA: 56s - loss: 2.4847 - acc: 0.087 - ETA: 47s - loss: 2.4849 - acc: 0.08 - ETA: 40s - loss: 2.4849 - acc: 0.07 - ETA: 35s - loss: 2.4850 - acc: 0.07 - ETA: 31s - loss: 2.4850 - acc: 0.07 - ETA: 28s - loss: 2.4850 - acc: 0.08 - ETA: 25s - loss: 2.4849 - acc: 0.07 - ETA: 23s - loss: 2.4847 - acc: 0.08 - ETA: 21s - loss: 2.4847 - acc: 0.08 - ETA: 20s - loss: 2.4848 - acc: 0.08 - ETA: 18s - loss: 2.4847 - acc: 0.08 - ETA: 17s - loss: 2.4847 - acc: 0.08 - ETA: 16s - loss: 2.4847 - acc: 0.08 - ETA: 15s - loss: 2.4847 - acc: 0.08 - ETA: 14s - loss: 2.4847 - acc: 0.08 - ETA: 13s - loss: 2.4847 - acc: 0.08 - ETA: 12s - loss: 2.4847 - acc: 0.09 - ETA: 11s - loss: 2.4846 - acc: 0.09 - ETA: 10s - loss: 2.4846 - acc: 0.09 - ETA: 10s - los

(1518, 224, 224, 3)